In [1]:
pip install matplotlib

Note: you may need to restart the kernel to use updated packages.


In [60]:
# Import Library yang dibutuhkan
import keras_tuner as kt

import pandas as pd
import numpy as np
import random

import tensorflow as tf
from tensorflow.keras.layers import Dense
from tensorflow.keras.models import Sequential,Model
from tensorflow.keras.utils import plot_model
from matplotlib import pyplot as plt

from sklearn.model_selection import train_test_split
from sklearn.preprocessing import RobustScaler,StandardScaler,OneHotEncoder,OrdinalEncoder

SEED_VALUE=123 #menetapkan seed value agar setiap kali kode jalan, hasilnya konsisten karena nilainya tidak akan acak setiap re-run code
random.seed(SEED_VALUE) 
np.random.seed(SEED_VALUE)

In [3]:
print('num Device: ',len(tf.config.experimental.list_physical_devices())) #cek device, apakah GPU/CPU sudah connect/belum
print(tf.config.experimental.list_physical_devices())

num Device:  2
[PhysicalDevice(name='/physical_device:CPU:0', device_type='CPU'), PhysicalDevice(name='/physical_device:GPU:0', device_type='GPU')]


# READING DATA

### data yang saya ambil adalah data yang memprediksi apakah seseorang mempunyai diabetes/tidak berdasarkan ciri ciri/features yang ada

In [4]:
# code ini menggunakan library pandas untuk read dataset csv
df = pd.read_csv('diabetes_prediction_dataset.csv') 
df.head() #menampilkan 5 data teratas

,gender,age,hypertension,heart_disease,smoking_history,bmi,HbA1c_level,blood_glucose_level,diabetes
0,Female,80.0,0,1,never,25.19,6.6,140,0
1,Female,54.0,0,0,No Info,27.32,6.6,80,0
2,Male,28.0,0,0,never,27.32,5.7,158,0
3,Female,36.0,0,0,current,23.45,5.0,155,0
4,Male,76.0,1,1,current,20.14,4.8,155,0


# Preprocessing

In [5]:
# menampilkan seluruh kolom + jumlah row, melihat tipe data tiap kolom, sekaligus melihat apakah ada missing value/tidak
df.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 100000 entries, 0 to 99999
Data columns (total 9 columns):
 #   Column               Non-Null Count   Dtype  
---  ------               --------------   -----  
 0   gender               100000 non-null  object 
 1   age                  100000 non-null  float64
 2   hypertension         100000 non-null  int64  
 3   heart_disease        100000 non-null  int64  
 4   smoking_history      100000 non-null  object 
 5   bmi                  100000 non-null  float64
 6   HbA1c_level          100000 non-null  float64
 7   blood_glucose_level  100000 non-null  int64  
 8   diabetes             100000 non-null  int64  
dtypes: float64(3), int64(4), object(2)
memory usage: 6.9+ MB


In [6]:
df.shape #melihat dimensi data (row, col)

(100000, 9)

#### dari info diatas, bisa dilihat dari total 100000 entries, tidak ada sama sekali kolom yang mengandung null values 

In [7]:
df.duplicated().sum() #melihat kolom yang duplikat = kolom duplikat berarti row nya sama persis dengan row lain

3854

In [8]:
df.drop_duplicates(inplace=True) #drop kolom duplicate
df.duplicated().sum() #cek lagi duplikatnya

0

### Split Columns

In [9]:
num_cols = [] #list utk numerical cols
cat_cols = [] #list utk categorical cols
 
for col in df.columns: #untuk setiap kolom di dataframe
    if df[col].dtype == 'object': #jika dia tipe datanya object, masuk (append) ke dalam cat_cols
        cat_cols.append(col)
    else: #jika bukan, maka masuk ke num_cols
        num_cols.append(col)

In [10]:
#hypertension dan heart_disease itu merupakan kolom yang sudah di encode, yang berarti merupakan sebuah categorical data, maka dari itu ditambahkan ke cat_cols
cat_cols.extend(['hypertension', 'heart_disease'])
for col in ['hypertension', 'heart_disease']: #jangan lupa hapus di num_cols
    num_cols.remove(col)

In [11]:
print(num_cols)
print(cat_cols)

['age', 'bmi', 'HbA1c_level', 'blood_glucose_level', 'diabetes']
['gender', 'smoking_history', 'hypertension', 'heart_disease']


##### cek unique kolom kategorik

In [12]:
for col in cat_cols: #untuk setiap kolom di categorical kolom
    print(df[col].unique()) #cari unique value, supaya kalo misalkan ada Typo atau inkonsistensi dalam penamaan value, bisa di-replace

['Female' 'Male' 'Other']
['never' 'No Info' 'current' 'former' 'ever' 'not current']
[0 1]
[1 0]


##### disini saya asumsikan bahwa ever dan not current adalah bagian dari former

In [13]:
#looping untuk me-replace ever dan not current dengan former
for i in ['ever', 'not current']:
    df['smoking_history'] = df['smoking_history'].replace({'ever': 'former', 'not current': 'former'}) 

In [14]:
df['smoking_history'].value_counts() #kembali cek unique valuenya

smoking_history
never      34398
No Info    32887
former     19664
current     9197
Name: count, dtype: int64

### Splitting Features and Target

In [15]:
x = df.drop(['diabetes'],axis=1) #memisahkan X (features) dan Y (target)
y = df['diabetes']

In [16]:
x

,gender,age,hypertension,heart_disease,smoking_history,bmi,HbA1c_level,blood_glucose_level
0,Female,80.0,0,1,never,25.19,6.6,140
1,Female,54.0,0,0,No Info,27.32,6.6,80
2,Male,28.0,0,0,never,27.32,5.7,158
3,Female,36.0,0,0,current,23.45,5.0,155
4,Male,76.0,1,1,current,20.14,4.8,155
...,...,...,...,...,...,...,...,...
99994,Female,36.0,0,0,No Info,24.60,4.8,145
99996,Female,2.0,0,0,No Info,17.37,6.5,100
99997,Male,66.0,0,0,former,27.83,5.7,155
99998,Female,24.0,0,0,never,35.42,4.0,100


In [17]:
y.head()

0    0
1    0
2    0
3    0
4    0
Name: diabetes, dtype: int64

In [18]:
class_distribution = y.value_counts() #menghitung distribusi Y sebagai target variable
print(class_distribution)

#bisa terlihat bahwa distribusinya tidak merata / imbalance, dimana orang yang tidak diabetes lebih banyak

diabetes
0    87664
1     8482
Name: count, dtype: int64


# Split Train, Test, Validation data

In [19]:
#split data x dan y menjadi x train test dan y train test dengan proporsi 80% training dan 20% testing, random state = SEED_VALUE yang berarti data yang di split selalu konsisten
x_train,x_test,y_train,y_test = train_test_split(x,y, test_size = 0.2,random_state = SEED_VALUE)

#split data training tadi yang 80%, lalu split menjadi x_train x_val dan y_train y_val, jadi 75% dari 80% tersebut akan digunakan untuk training sebenarnya, dan 25% untuk validasi
x_train,x_val,y_train,y_val = train_test_split(x_train,y_train, test_size = 0.25,random_state = SEED_VALUE) 
print(x_train.shape,y_train.shape)
print(x_val.shape,y_val.shape)
print(x_test.shape,y_test.shape)

(57687, 8) (57687,)
(19229, 8) (19229,)
(19230, 8) (19230,)


##### setelah split train test dan val data, sekarang saya ingin melakukan preprocessing lebih lanjut yaitu encoding & scaling

In [20]:
ohe_encoder = OneHotEncoder(sparse_output=False, handle_unknown='ignore') #menggunakan ohe encoding karena valuenya itu lumayan banyak sekitar 3-4 dan tidak berurutan (ordinal) sehingga OHE pilihan yang tepat
ohe_cols = ['gender', 'smoking_history']  # Hanya encode kolom ini

# karena hyperextension dan heart_disease udh encoded, 
# berarti hanya perlu encode sisanya yang belum yaitu gender dan smoking history

#melakukan proses OHE encoding ke x_train, x_val dan X_test
new_cat_cols = ohe_encoder.fit_transform(x_train[ohe_cols])
encoded_data = pd.DataFrame(new_cat_cols, columns=ohe_encoder.get_feature_names_out(ohe_cols))
x_train = x_train.reset_index(drop=True)
x_train = pd.concat([x_train.drop(ohe_cols, axis=1, errors='ignore'), encoded_data], axis=1)

new_cat_cols = ohe_encoder.transform(x_val[ohe_cols])
encoded_data = pd.DataFrame(new_cat_cols, columns=ohe_encoder.get_feature_names_out(ohe_cols))
x_val = x_val.reset_index(drop=True)
x_val = pd.concat([x_val.drop(ohe_cols,axis=1,errors='ignore'),encoded_data],axis=1)

new_cat_cols = ohe_encoder.transform(x_test[ohe_cols])
encoded_data = pd.DataFrame(new_cat_cols, columns=ohe_encoder.get_feature_names_out(ohe_cols))
x_test = x_test.reset_index(drop=True)
x_test = pd.concat([x_test.drop(ohe_cols,axis=1,errors='ignore'),encoded_data],axis=1)

In [21]:
x_test

,age,hypertension,heart_disease,bmi,HbA1c_level,blood_glucose_level,gender_Female,gender_Male,gender_Other,smoking_history_No Info,smoking_history_current,smoking_history_former,smoking_history_never
0,66.0,0,0,28.06,4.8,130,1.0,0.0,0.0,1.0,0.0,0.0,0.0
1,34.0,0,0,27.32,6.2,126,0.0,1.0,0.0,0.0,0.0,0.0,1.0
2,49.0,0,0,21.14,5.7,85,1.0,0.0,0.0,1.0,0.0,0.0,0.0
3,53.0,0,0,30.06,6.1,140,0.0,1.0,0.0,1.0,0.0,0.0,0.0
4,28.0,0,0,25.37,4.5,130,1.0,0.0,0.0,1.0,0.0,0.0,0.0
...,...,...,...,...,...,...,...,...,...,...,...,...,...
19225,80.0,1,0,27.32,3.5,126,0.0,1.0,0.0,1.0,0.0,0.0,0.0
19226,72.0,1,0,26.26,6.0,145,0.0,1.0,0.0,1.0,0.0,0.0,0.0
19227,5.0,0,0,16.40,5.0,160,1.0,0.0,0.0,1.0,0.0,0.0,0.0
19228,53.0,0,0,24.15,4.5,145,0.0,1.0,0.0,1.0,0.0,0.0,0.0


In [22]:
#drop kolom gender Other dan smoking_history no info karena mereka merupakan data yang kosong/tidak ada informasi, dengan OHE, maka data kosong tersebut bisa diwakilkan di kolom yang tersisa jika semua kolom bernilai 0
x_test.drop(columns=['gender_Other', 'smoking_history_No Info'], inplace=True)
x_val.drop(columns=['gender_Other', 'smoking_history_No Info'], inplace=True)
x_train.drop(columns=['gender_Other', 'smoking_history_No Info'], inplace=True)

In [23]:
x_train

,age,hypertension,heart_disease,bmi,HbA1c_level,blood_glucose_level,gender_Female,gender_Male,smoking_history_current,smoking_history_former,smoking_history_never
0,30.0,0,0,31.30,6.5,140,0.0,1.0,0.0,0.0,1.0
1,13.0,0,0,29.09,5.8,200,1.0,0.0,0.0,0.0,0.0
2,75.0,0,1,27.32,5.7,90,0.0,1.0,0.0,0.0,0.0
3,52.0,0,0,27.32,6.6,160,0.0,1.0,0.0,0.0,0.0
4,14.0,0,0,22.82,6.0,130,0.0,1.0,0.0,0.0,1.0
...,...,...,...,...,...,...,...,...,...,...,...
57682,33.0,0,0,27.50,5.7,85,0.0,1.0,0.0,0.0,1.0
57683,16.0,0,0,24.59,6.6,100,0.0,1.0,0.0,0.0,0.0
57684,46.0,0,0,32.98,3.5,159,0.0,1.0,0.0,0.0,1.0
57685,80.0,0,0,27.32,5.7,140,1.0,0.0,0.0,0.0,0.0


In [24]:
num_cols.remove('diabetes') #sebelum kita scaling, kita remove dulu target variable karena berupa categorical 0 dan 1

In [25]:
scaler = StandardScaler() #menggunakan standard scaler untuk merubah angka menjadi lebih mudah dibaca oleh komputer (berlaku ke data train, val, test)
x_train[num_cols] = scaler.fit_transform(x_train[num_cols])
x_val[num_cols] = scaler.transform(x_val[num_cols])
x_test[num_cols] = scaler.transform(x_test[num_cols])

# build model

In [26]:
#mulai membuat model, disini menggabungkan data training, testing, val bagi x dan y menjadi satu kesatuan tensor dataset dengan batch pembagian data 32 dan shuffle yang berarti mengacak urutan data, sehingga tidak mengambil data yang berurutan karena berpotensi bias, apalagi untuk training
train_ds = tf.data.Dataset.from_tensor_slices((x_train,y_train)).batch(32).shuffle(10)
test_ds = tf.data.Dataset.from_tensor_slices((x_test,y_test)).batch(32)
val_ds = tf.data.Dataset.from_tensor_slices((x_val,y_val)).batch(32)

In [27]:
val_ds #dengan memanggil salah satu variable tensor dataset, kita bisa liat shapenya, yang merupakan jumlah kolom untuk digunakan di model

<ShuffleDataset element_spec=(TensorSpec(shape=(None, 11), dtype=tf.float64, name=None), TensorSpec(shape=(None,), dtype=tf.int64, name=None))>

### keras sequential

In [45]:
model = tf.keras.Sequential([ #menggunakan model keras sequential
    Dense(12, activation="relu",input_shape=(11,)), #disini saya menggunakan neuron 12, tidak terlalu banyak karena datanya memang tidak terlalu kompleks.
    #menggunakan relu activation dan input shape (11 kolom)
    Dense(4, activation="relu"),
    Dense(1, activation='sigmoid'), #untuk output 1 yang berarti binary classification dan sigmoid merupakan activation function untuk klasifikasi biner
])

model.summary() #melihat summary dari model

Model: "sequential_1"
_________________________________________________________________
 Layer (type)                Output Shape              Param #   
 dense_6 (Dense)             (None, 12)                144       
                                                                 
 dense_7 (Dense)             (None, 4)                 52        
                                                                 
 dense_8 (Dense)             (None, 1)                 5         
                                                                 
Total params: 201
Trainable params: 201
Non-trainable params: 0
_________________________________________________________________


### functional API

In [46]:
inputs = tf.keras.Input(shape=(11,)) #model functional API --> biasanya digunakan lebih sering karena dia fleksibel untuk data kompleks atau simple
dense1 = Dense(12, activation="relu")(inputs) #functional API bekerja dengan proses menghubungkan, jadi dimulai dari input, lalu input nyambung dengan dense1, dan seterusnya
dense2 = Dense(4, activation='relu')(dense1) #neuron yang saya pakai juga sama dengan keras sequential untuk membandingkan hasil
outputs = Dense(1, activation='sigmoid')(dense2) #output binary, jadi menggunakan sigmoid

model_functional = Model(inputs=inputs, outputs=outputs)
model_functional.summary() #summary dari model functional API

Model: "model_1"
_________________________________________________________________
 Layer (type)                Output Shape              Param #   
 input_2 (InputLayer)        [(None, 11)]              0         
                                                                 
 dense_9 (Dense)             (None, 12)                144       
                                                                 
 dense_10 (Dense)            (None, 4)                 52        
                                                                 
 dense_11 (Dense)            (None, 1)                 5         
                                                                 
Total params: 201
Trainable params: 201
Non-trainable params: 0
_________________________________________________________________


# Train Model

In [62]:
from tensorflow.keras.metrics import Precision, Recall, AUC #setelah model sudah di-compile, sekarang waktunya training data
#saya menggunakan beberapa metric yaitu precision, recall, AUC, dan accuracy

#precision penting karena ingin memprediksi diabetes dan kita ingin meminimalkan false positives yaitu kita mau memastikan kalo orang itu memang punya diabetes, maka kita bisa melakukan pengobatan. Hal ini membuat ketepatan prediksi semakin tinggi
#recall mengukur seberapa baik model mendeteksi semua kasus positif. Dalam konteks diabetes, kita mau memastika bahwa sebanyak mungkin kasus positif terdeteksi, false negatives (orang yang terkena diabetes, dianggap tidak diabetes) sangat berbahaya
#menggunakan AUC (area under curve) karena membantu mengevaluasi kemampuan model dalam membedakan mana positif dan negatif. Ini metrik yang sesuai untuk menangani data imbalance.
#ROC Curve itu adalah kurva yang merupakan hubungan antara True Positive dan False positive, yang dimana AUC, mengukur luas area di bawah kurva ROC. AUC memberikan gambaran seberapa baik model dalam bedain kelas positif dan negatif di ROC curve itu.
#Jika AUC = 1. maka model sempurna dalam bedain kelas positif dan negatif, berarti akurasi bagus
#jika AUC = 0.5 maka model tidak bisa membedakan dengan baik
#Jika AUC < 0.5 ini berarti model buruk, karena dia memprediksinya terbaik (yang positif diprediksi negative, vice versa)

### apa itu callback function?
### callback function itu function yang di-pass ke function lain sebagai argumen dan dapat dipanggil lagi pada saat tertentu. Nah panggilnya itu biasanya saat saat tertentu saja ketika suatu goal/kondisi sudah tercapai. Misalnya, early stopping sebagai salah 1 implementasi callbacks function
### Contoh implementasinya itu, saat membuat function earlyStopping, function tersebut akan dijadikan argumen di fitting model, nah setelah goal/kondisi tercapai, yaitu ketika tidak ada perkembangan saat training sebanyak patience (5x), maka callbacks akan dijalankan / dipanggil dan akan memberhentikan proses training

In [48]:
from tensorflow.keras.callbacks import EarlyStopping #implementasi Callbacks function (Early Stopping)
early_stopping = EarlyStopping(monitor='val_loss', patience=5, restore_best_weights=True) 
#early stopping oleh callbacks function itu salah satu fungsi yang dapat memberhentikan proses model fitting jika tidak adanya peningkatan lagi. 
#contoh jika 10 epoch dijalankan dan di epoch ke 3 sudah mulai tidak ada peningkatan, jika berlangsung sebanyak parameter patience (contoh 5x gaada perubahan) maka model akan secara otomatis berhenti
#ini mencegah overfitting dan menghemat waktu

### Train model sequential

In [49]:
model.compile(optimizer=tf.keras.optimizers.Adam(learning_rate=0.01), #melakukan training sequential dengan adam sebagai optimizer, learning rate 0.01
              loss='binary_crossentropy', #loss function binary_crossentropy sebagai loss function utk klasifikasi biner
              metrics=['accuracy', 'Precision', 'Recall', 'AUC']) #disini menggunakan metrics accuracy, precision, recall, AUC

In [50]:
fitting_sequential = model.fit(train_ds, validation_data=val_ds, epochs=10, callbacks=[early_stopping]) #fitting model
#10 epoch
#menggunakan early_stopping sebagai argumen

Epoch 1/10
1803/1803 [==============================] - 6s 3ms/step - loss: 0.1207 - accuracy: 0.9566 - precision: 0.8563 - recall: 0.6169 - auc: 0.9573 - val_loss: 0.0966 - val_accuracy: 0.9666 - val_precision: 0.9487 - val_recall: 0.6558 - val_auc: 0.9701
Epoch 2/10
1803/1803 [==============================] - 5s 3ms/step - loss: 0.0957 - accuracy: 0.9667 - precision: 0.9653 - recall: 0.6492 - auc: 0.9710 - val_loss: 0.0917 - val_accuracy: 0.9674 - val_precision: 0.9935 - val_recall: 0.6334 - val_auc: 0.9752
Epoch 3/10
1803/1803 [==============================] - 5s 3ms/step - loss: 0.0909 - accuracy: 0.9680 - precision: 0.9771 - recall: 0.6558 - auc: 0.9733 - val_loss: 0.0878 - val_accuracy: 0.9685 - val_precision: 0.9665 - val_recall: 0.6653 - val_auc: 0.9746
Epoch 4/10
1803/1803 [==============================] - 5s 3ms/step - loss: 0.0895 - accuracy: 0.9689 - precision: 0.9830 - recall: 0.6624 - auc: 0.9739 - val_loss: 0.0934 - val_accuracy: 0.9653 - val_precision: 0.9812 - val_r

### Train model functional

In [51]:
model_functional.compile( #compile model functional
    optimizer=tf.keras.optimizers.Adam(learning_rate=0.01),  #utk functional sama saja parameter dengan sequential
    loss='binary_crossentropy',
    metrics=['accuracy', 'Precision', 'Recall', 'AUC']
)

In [52]:
fitting_functional = model_functional.fit(train_ds,validation_data=val_ds,epochs = 10, callbacks=[early_stopping]) #fitting model

Epoch 1/10
1803/1803 [==============================] - 6s 3ms/step - loss: 0.1177 - accuracy: 0.9588 - precision: 0.8947 - recall: 0.6098 - auc: 0.9590 - val_loss: 0.0936 - val_accuracy: 0.9670 - val_precision: 0.9648 - val_recall: 0.6487 - val_auc: 0.9726
Epoch 2/10
1803/1803 [==============================] - 5s 3ms/step - loss: 0.0922 - accuracy: 0.9680 - precision: 0.9774 - recall: 0.6556 - auc: 0.9725 - val_loss: 0.0877 - val_accuracy: 0.9691 - val_precision: 0.9757 - val_recall: 0.6647 - val_auc: 0.9746
Epoch 3/10
1803/1803 [==============================] - 5s 3ms/step - loss: 0.0902 - accuracy: 0.9687 - precision: 0.9851 - recall: 0.6583 - auc: 0.9732 - val_loss: 0.0879 - val_accuracy: 0.9695 - val_precision: 0.9955 - val_recall: 0.6564 - val_auc: 0.9753
Epoch 4/10
1803/1803 [==============================] - 5s 3ms/step - loss: 0.0888 - accuracy: 0.9691 - precision: 0.9909 - recall: 0.6587 - auc: 0.9740 - val_loss: 0.0864 - val_accuracy: 0.9699 - val_precision: 0.9973 - val_r

### performa overall dari kedua model bisa dibilang mirip, hanya sedikit perbedaan yang terlihat dari beberapa metric. Model functional APi itu lebih tinggi di bagian recall, sedangkan sequential di bagian precision. Untuk akurasi dan AUC kedua model hampir identik. Kesimpulannya, kedua model sudah sangat baik untuk memprediksi dataset ini.

# Testing model

### Sequential

In [58]:
model.evaluate(test_ds)

601/601 [==============================] - 1s 2ms/step - loss: 0.0818 - accuracy: 0.9719 - precision: 1.0000 - recall: 0.6731 - auc: 0.9760


[0.08175625652074814,
 0.9719188809394836,
 1.0,
 0.6731234788894653,
 0.9760300517082214]

### Functional

In [59]:
model_functional.evaluate(test_ds)

601/601 [==============================] - 1s 2ms/step - loss: 0.0817 - accuracy: 0.9724 - precision: 0.9947 - recall: 0.6822 - auc: 0.9756


[0.08166327327489853,
 0.97238689661026,
 0.9947043061256409,
 0.6822034120559692,
 0.9755522012710571]

### untuk bagian hasil testing juga overall sama kedua model, mungkin akan terlihat berbeda di dataset yang lebih kompleks.
### Dari kedua model tersebut, hanya terlihat perbedaan di bagian recall dan precision
### precision pada Function API model lebih tinggi 1% dan recall dan AUC pada sequential model lebih tinggi 0.5%
### untuk metric lain seperti loss dan accuracy tidak terlihat perbedaan yang signifikan

# Hyperparameter Tuning

In [55]:
# from keras.wrappers.scikit_learn import KerasClassifier
# from sklearn.model_selection import GridSearchCV

# def create_model_sequential(optimizer='adam', activation='relu', neurons=64):
#     model = Sequential()
#     model.add(Dense(neurons, activation=activation, input_shape=(11,)))
#     model.add(Dense(neurons, activation=activation))
#     model.add(Dense(1, activation='sigmoid'))  # Output layer for binary classification
#     model.compile(optimizer=optimizer, loss='binary_crossentropy', metrics=['accuracy'])
#     return model

# param_grid_sequential = {
#     'optimizer': ['adam', 'sgd'],
#     'activation': ['relu', 'sigmoid', 'tanh'],
#     'neurons': [12, 32, 64],
#     'epochs': [10, 20],
#     'batch_size': [32, 64]
# }

# model_sequential = KerasClassifier(build_fn=create_model_sequential)

# grid_search_sequential = GridSearchCV(estimator=model_sequential, param_grid=param_grid_sequential, cv=3)
# grid_search_result_sequential = grid_search_sequential.fit(x_train, y_train)

# print("Best Parameters for Sequential Model:", grid_search_result_sequential.best_params_)